In [1]:
# Install dependencies
!pip install ultralytics -q
!pip install git+https://github.com/openai/CLIP.git -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.7 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.0 MB/s eta 0:00:00


In [2]:
# ============================================================
# OPTIMIZED VIDEOPATCHCORE - SPEED MAXIMIZED
# ============================================================
# Key optimizations:
# 1. Batched flip detection (process multiple frames at once)
# 2. Cached YOLO detections
# 3. Batched feature extraction
# 4. Faster coreset with random pre-filtering
# 5. Reduced I/O with smart caching
# 6. Mixed precision throughout
# ============================================================

import numpy as np
import os
import shutil
from tqdm import tqdm
import random
from glob import glob
import warnings
warnings.filterwarnings("ignore")

from PIL import Image
import matplotlib.pyplot as plt
import cv2
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
import clip

# Seeds
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

if device.type == 'cuda':
    torch.backends.cudnn.benchmark = True
    print(f"GPU: {torch.cuda.get_device_name(0)}")

# ============================================================
# PATHS & CONFIG
# ============================================================
TRAIN_PATH = '/kaggle/input/pixel-play-26/Avenue_Corrupted-20251221T112159Z-3-001/Avenue_Corrupted/Dataset/training_videos'
TEST_PATH_ORIGINAL = '/kaggle/input/pixel-play-26/Avenue_Corrupted-20251221T112159Z-3-001/Avenue_Corrupted/Dataset/testing_videos'
CLEAN_TEST_PATH = '/kaggle/working/cleaned_testing_videos'
OUTPUT_FILE = "submission.csv"

# VPC Config
CLIP_LEN = 10
CNL_POOL = 32
OBJ_SIZE = (64, 64)
IMG_SIZE = (224, 224)
POWER_N = 4
POOL_FOR_SM = 4
FACTOR_X = 1.5
FACTOR_Y = 1.2
BATCH_SIZE = 1
NUM_WORKERS = 4

# Flip detector config
FLIP_IMG_SIZE = 64
FLIP_BATCH_SIZE = 128  # Increased for speed
FLIP_EPOCHS = 5        # Reduced - usually converges fast

print(f"Train path exists: {os.path.exists(TRAIN_PATH)}")
print(f"Test path exists: {os.path.exists(TEST_PATH_ORIGINAL)}")

Device: cuda
GPU: Tesla T4
Train path exists: True
Test path exists: True


In [3]:
# ============================================================
# FAST FLIP DETECTOR
# ============================================================

class FlipDetectionDataset(Dataset):
    """Optimized flip detection dataset"""
    def __init__(self, video_path, img_size=64, samples_per_class=2000):  # Reduced samples
        self.img_size = img_size
        self.transform = transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.ToTensor(),
        ])
        
        # Collect frame paths efficiently
        self.frame_paths = []
        for folder in os.listdir(video_path):
            folder_path = os.path.join(video_path, folder)
            if os.path.isdir(folder_path):
                frames = [os.path.join(folder_path, f) for f in os.listdir(folder_path) 
                         if f.endswith(('.jpg', '.png'))]
                self.frame_paths.extend(frames)
        
        print(f"Total frames: {len(self.frame_paths)}")
        
        # Sample
        selected = random.sample(self.frame_paths, min(samples_per_class, len(self.frame_paths)))
        self.samples = [(p, 0) for p in selected] + [(p, 1) for p in selected]
        random.shuffle(self.samples)
        print(f"Training samples: {len(self.samples)}")
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        path, label = self.samples[idx]
        img = Image.open(path).convert('L')
        tensor = self.transform(img)
        if label == 1:
            tensor = TF.vflip(tensor)
        return tensor, torch.tensor(label, dtype=torch.long)


class FlipDetectorCNN(nn.Module):
    """Lightweight flip detector"""
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 16, 3, stride=2, padding=1), nn.ReLU(),
            nn.Conv2d(16, 32, 3, stride=2, padding=1), nn.ReLU(),
            nn.Conv2d(32, 64, 3, stride=2, padding=1), nn.ReLU(),
            nn.AdaptiveAvgPool2d(1),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64, 32), nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(32, 2)
        )
    
    def forward(self, x):
        return self.classifier(self.features(x))


def train_flip_detector():
    """Train flip detector with speed optimizations"""
    print("\n" + "="*60)
    print("Training Flip Detector (Optimized)")
    print("="*60)
    
    dataset = FlipDetectionDataset(TRAIN_PATH, FLIP_IMG_SIZE, samples_per_class=2000)
    loader = DataLoader(dataset, batch_size=FLIP_BATCH_SIZE, shuffle=True, 
                       num_workers=4, pin_memory=True)
    
    model = FlipDetectorCNN().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scaler = torch.amp.GradScaler('cuda')  # Mixed precision
    
    for epoch in range(FLIP_EPOCHS):
        model.train()
        correct, total = 0, 0
        
        pbar = tqdm(loader, desc=f'Epoch {epoch+1}/{FLIP_EPOCHS}')
        for images, labels in pbar:
            images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            
            with torch.amp.autocast('cuda'):
                outputs = model(images)
                loss = criterion(outputs, labels)
            
            optimizer.zero_grad(set_to_none=True)  # Faster than zero_grad()
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            correct += (outputs.argmax(1) == labels).sum().item()
            total += labels.size(0)
            pbar.set_postfix({'acc': f'{100*correct/total:.1f}%'})
        
        print(f"Epoch {epoch+1}: Accuracy = {100*correct/total:.1f}%")
    
    return model


def fix_flipped_frames_batched(model, src_path, dst_path, batch_size=64):
    """Batched flip detection and correction - MUCH faster"""
    print("\n" + "="*60)
    print("Fixing Flipped Frames (Batched)")
    print("="*60)
    
    model.eval()
    transform = transforms.Compose([
        transforms.Resize((FLIP_IMG_SIZE, FLIP_IMG_SIZE)),
        transforms.ToTensor()
    ])
    
    if os.path.exists(dst_path):
        shutil.rmtree(dst_path)
    os.makedirs(dst_path)
    
    flipped_count = 0
    total_count = 0
    
    video_folders = sorted([f for f in os.listdir(src_path) if os.path.isdir(os.path.join(src_path, f))])
    
    for folder in tqdm(video_folders, desc='Processing videos'):
        src_folder = os.path.join(src_path, folder)
        dst_folder = os.path.join(dst_path, folder)
        os.makedirs(dst_folder, exist_ok=True)
        
        frames = sorted([f for f in os.listdir(src_folder) if f.endswith(('.jpg', '.png'))])
        
        # Process in batches
        all_paths = [os.path.join(src_folder, f) for f in frames]
        all_dst_paths = [os.path.join(dst_folder, f) for f in frames]
        flip_flags = []
        
        for i in range(0, len(all_paths), batch_size):
            batch_paths = all_paths[i:i+batch_size]
            
            # Load batch
            batch_tensors = []
            for p in batch_paths:
                img = Image.open(p).convert('L')
                batch_tensors.append(transform(img))
            
            batch = torch.stack(batch_tensors).to(device, non_blocking=True)
            
            with torch.no_grad(), torch.amp.autocast('cuda'):
                outputs = model(batch)
                probs = torch.softmax(outputs, dim=1)[:, 1]
                is_flipped = (probs > 0.5).cpu().tolist()
            
            flip_flags.extend(is_flipped)
        
        # Save frames (copy or flip)
        for src_p, dst_p, is_flip in zip(all_paths, all_dst_paths, flip_flags):
            if is_flip:
                img = Image.open(src_p)
                img.transpose(Image.FLIP_TOP_BOTTOM).save(dst_p)
                flipped_count += 1
            else:
                shutil.copy2(src_p, dst_p)
            total_count += 1
    
    print(f"Total: {total_count}, Corrected: {flipped_count} ({100*flipped_count/total_count:.1f}%)")
    return flipped_count

In [4]:
# ============================================================
# RUN FLIP DETECTION
# ============================================================
flip_model = train_flip_detector()
fix_flipped_frames_batched(flip_model, TEST_PATH_ORIGINAL, CLEAN_TEST_PATH)

TEST_PATH = CLEAN_TEST_PATH
print(f"\n*** TEST_PATH set to: {TEST_PATH} ***")


Training Flip Detector (Optimized)
Total frames: 9204
Training samples: 4000


Epoch 1/5: 100%|██████████| 32/32 [00:10<00:00,  3.19it/s, acc=51.6%]


Epoch 1: Accuracy = 51.6%


Epoch 2/5: 100%|██████████| 32/32 [00:04<00:00,  6.43it/s, acc=69.6%]


Epoch 2: Accuracy = 69.6%


...

Epoch 4: Accuracy = 99.9%


Epoch 5/5: 100%|██████████| 32/32 [00:05<00:00,  6.07it/s, acc=100.0%]


Epoch 5: Accuracy = 100.0%

Fixing Flipped Frames (Batched)


Processing videos: 100%|██████████| 21/21 [02:13<00:00,  6.33s/it]

Total: 11706, Corrected: 1195 (10.2%)

*** TEST_PATH set to: /kaggle/working/cleaned_testing_videos ***


In [5]:
# ============================================================
# YOLO SETUP WITH CACHING
# ============================================================

print("Loading YOLOv5...")
yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5l', pretrained=True, trust_repo=True, verbose=False)
yolo_model.eval()
print("YOLOv5L loaded.")

# Detection cache for speed
DETECTION_CACHE = {}

def get_resized_area(areas, max_w, max_h, factor_x=FACTOR_X, factor_y=FACTOR_Y):
    """Resize bounding boxes with margin"""
    if factor_x == 0 and factor_y == 0:
        return areas
    new_areas = []
    for area in areas:
        xmin_, ymin_, xmax_, ymax_ = area[:4]
        xmin = xmin_ - (factor_x - 1) * (xmax_ - xmin_)
        ymin = ymin_ - (factor_y - 1) * (ymax_ - ymin_)
        xmax = xmax_ + (factor_x - 1) * (xmax_ - xmin_)
        ymax = ymax_ + (factor_y - 1) * (ymax_ - ymin_)
        
        # Make square
        x, y = xmax - xmin, ymax - ymin
        if y > x:
            dif = (y - x) / 2
            xmax += dif
            xmin -= dif
        elif y < x:
            dif = (x - y) / 2
            ymax += dif
            ymin -= dif
        
        xmin, ymin = max(0, xmin), max(0, ymin)
        xmax, ymax = min(max_w, xmax), min(max_h, ymax)
        new_areas.append([xmin, ymin, xmax, ymax])
    return new_areas


def detect_objects_cached(frame_path, confidence=0.4, coi=[0]):
    """Cached YOLO detection"""
    if frame_path in DETECTION_CACHE:
        return DETECTION_CACHE[frame_path]
    
    with torch.no_grad():
        results = yolo_model(frame_path)
    
    areas = results.xyxy[0].cpu().numpy()
    filtered = [area for area in areas if area[4] > confidence and area[-1] in coi]
    
    DETECTION_CACHE[frame_path] = filtered
    return filtered

Loading YOLOv5...
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


YOLOv5 🚀 2026-1-7 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)

100%|██████████| 89.3M/89.3M [00:00<00:00, 222MB/s]

Fusing layers... 
YOLOv5l summary: 367 layers, 46533693 parameters, 0 gradients, 109.0 GFLOPs
Adding AutoShape... 


YOLOv5L loaded.


In [6]:
# ============================================================
# NORMALIZATION (Preserving exact augmentation logic)
# ============================================================

def vpc_normalize_cv2(img, target_h, target_w, augment=False):
    """Normalize with optional noise augmentation"""
    img = cv2.resize(img, (target_w, target_h)).astype('float32')
    img = img / 255.0

    # Noise augmentation (40% probability) - PRESERVED
    if augment and np.random.rand() < 0.4:
        noise_factor = 0.1373
        noise = np.random.uniform(-noise_factor, noise_factor, img.shape)
        img = np.clip(img + noise, 0.0, 1.0)

    # CLIP normalization
    mean = np.array([0.48145466, 0.4578275, 0.40821073]).reshape(1, 1, 3)
    std = np.array([0.26862954, 0.26130258, 0.27577711]).reshape(1, 1, 3)
    img = (img - mean) / std

    img = np.transpose(img, [2, 0, 1])
    return torch.from_numpy(img).float()


def load_frame_cv2(path, h=224, w=224, augment=False):
    """Load and normalize frame"""
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return vpc_normalize_cv2(img, h, w, augment=augment)


def crop_and_normalize(img_cv2, bbox, obj_h=64, obj_w=64, augment=False):
    """Crop and normalize object"""
    xmin, ymin, xmax, ymax = [int(x) for x in bbox]
    cropped = img_cv2[ymin:ymax, xmin:xmax, :]
    return vpc_normalize_cv2(cropped, obj_h, obj_w, augment=augment)

In [7]:
# ============================================================
# POOLING FUNCTIONS (Optimized)
# ============================================================

def ChannelAvgPool2d(tensor, out_channel=32):
    """Channel-wise average pooling"""
    B, C, H, W = tensor.shape
    group_size = C // out_channel
    return tensor.view(B, out_channel, group_size, H, W).mean(dim=2)


def TemporalAvgPool3d(tensor, out_depths=1):
    """Temporal average pooling"""
    return F.adaptive_avg_pool3d(tensor, (out_depths, tensor.shape[3], tensor.shape[4]))


def TemporalMaxPool3d(tensor, out_depths=1):
    """Temporal max pooling"""
    return F.adaptive_max_pool3d(tensor, (out_depths, tensor.shape[3], tensor.shape[4]))


def SpatialAvgPool3d(tensor, out_hw=1):
    """Spatial average pooling"""
    d = tensor.shape[2]
    return F.adaptive_avg_pool3d(tensor, (d, out_hw, out_hw))


def SpatialMaxPool3d(tensor, out_hw=1):
    """Spatial max pooling"""
    d = tensor.shape[2]
    return F.adaptive_max_pool3d(tensor, (d, out_hw, out_hw))

In [8]:
# ============================================================
# VPC MODEL
# ============================================================

class VPCModel(nn.Module):
    def __init__(self, device=device, cnl_pool=CNL_POOL):
        super().__init__()
        clip_model, _ = clip.load("RN101", device=device)
        self.backbone = clip_model.visual.float()
        self.cnl_pool = cnl_pool
        
        for p in self.backbone.parameters():
            p.requires_grad = False
        self.backbone.eval()

        self.feature_maps = []
        self.backbone.layer2[-1].register_forward_hook(self.hook)
        self.backbone.layer3[-1].register_forward_hook(self.hook)
        
        self.avg = nn.AvgPool2d(3, stride=1)
        self.resize = nn.AdaptiveAvgPool2d(28)
        self.avg2 = nn.AvgPool2d(3, stride=1)
        self.resize2 = nn.AdaptiveAvgPool2d(14)

    def hook(self, module, input, output):
        self.feature_maps.append(output.float())

    @torch.no_grad()
    def forward(self, x):
        self.feature_maps = []
        _ = self.backbone(x)
        return self.feature_maps

    def make_locally_aware_feature(self, feature_maps, len_frame):
        """Create locally-aware features"""
        resized = [self.resize(self.avg(fmap)) for fmap in feature_maps]
        locally_feat = torch.cat(resized, 1)
        locally_feat = ChannelAvgPool2d(locally_feat, self.cnl_pool)
        chunked = torch.chunk(locally_feat, chunks=len_frame, dim=0)
        locally_feat = torch.stack(chunked, dim=0)
        return locally_feat.permute(1, 2, 0, 3, 4)

    def make_globally_aware_feature(self, feature_maps):
        """Create globally-aware features"""
        resized = [self.resize2(self.avg2(fmap)) for fmap in feature_maps]
        global_feat = torch.cat(resized, 1)
        global_feat = global_feat.permute(1, 0, 2, 3).unsqueeze(0)
        ap = nn.AvgPool3d((1, 14, 14))
        mp = nn.MaxPool3d((1, 14, 14))
        global_feat = ap(global_feat) + mp(global_feat)
        return global_feat.view(1, -1, global_feat.shape[2])

print("Loading CLIP RN101...")
vpc = VPCModel().to(device)
vpc.eval()
print("VPC Model loaded.")

Loading CLIP RN101...


100%|███████████████████████████████████████| 278M/278M [00:03<00:00, 80.6MiB/s]


VPC Model loaded.


In [9]:
# ============================================================
# FEATURE FUNCTIONS
# ============================================================

def MakeSpatialFeature(feature, kernel_size=(1, POOL_FOR_SM, POOL_FOR_SM)):
    """Create spatial features from local features"""
    len_channel = feature.shape[1]
    f1 = TemporalAvgPool3d(feature, 1)
    f2 = TemporalMaxPool3d(feature, 1)
    feature = f1 + f2
    if kernel_size != (1, 1, 1):
        d = feature.shape[2] // kernel_size[0]
        h = feature.shape[3] // kernel_size[1]
        w = feature.shape[4] // kernel_size[2]
        feature = F.adaptive_avg_pool3d(feature, (d, h, w))
    feature = feature.squeeze(2).permute(0, 2, 3, 1)
    return feature.reshape(-1, len_channel)


def MakeTemporalFeature(feature, mode='train'):
    """Create temporal (motion) features"""
    len_channel = feature.shape[1]
    len_frame = feature.shape[2]
    resolution = feature.shape[3]
    
    # Compute frame differences
    temporal = feature[:, :, 1:] - feature[:, :, :-1]
    temporal = torch.abs(temporal)
    
    if mode == 'train':
        temporal = SpatialAvgPool3d(temporal, 1)
    else:
        temporal = SpatialMaxPool3d(temporal, 1)
    
    temporal = temporal.permute(0, 2, 1, 3, 4)
    return temporal.reshape(-1, len_channel)


def MakeHighlevelFeature(feature):
    """Create high-level semantic features with temporal pyramid pooling"""
    pool = nn.MaxPool1d(2, stride=1)
    channels = feature.shape[1]
    f0 = feature
    f1 = pool(f0)
    f2 = pool(f1)
    out = F.adaptive_max_pool1d(f2, output_size=f1.shape[2])
    out = out + f1
    out = F.adaptive_max_pool1d(out, output_size=f0.shape[2])
    out = out + f0
    return out.view(1, channels, -1).permute(0, 2, 1).squeeze(0)

In [10]:
# ============================================================
# FAST CORESET SUBSAMPLING
# ============================================================

def get_coreset_fast(memory_bank, l=1000, device='cuda'):
    """Fast coreset with random pre-filtering"""
    if len(memory_bank) <= l:
        return memory_bank
    
    # Random pre-filter if too large (speed optimization)
    if len(memory_bank) > 15000:
        indices = torch.randperm(len(memory_bank))[:15000]
        memory_bank = memory_bank[indices]
    
    memory_bank = memory_bank.to(device)
    n = len(memory_bank)
    
    # Greedy coreset selection
    selected = [random.randint(0, n-1)]
    min_distances = torch.full((n,), float('inf'), device=device)
    
    for _ in tqdm(range(l - 1), desc='Coreset', leave=False):
        # Update distances
        last = memory_bank[selected[-1]:selected[-1]+1]
        dists = torch.linalg.norm(memory_bank - last, dim=1)
        min_distances = torch.minimum(dists, min_distances)
        min_distances[selected[-1]] = -1
        
        # Select farthest point
        selected.append(torch.argmax(min_distances).item())
    
    return memory_bank[selected].cpu()


def predict(features, memory_bank, power_n=POWER_N):
    """Compute anomaly score"""
    distances = torch.cdist(features, memory_bank, p=2.0)
    dist_score, _ = torch.min(distances, dim=1)
    s_star = torch.max(dist_score)
    return s_star ** power_n

In [11]:
# ============================================================
# DATASET & OBJECT EXTRACTION
# ============================================================

class ClipDataset(Dataset):
    def __init__(self, root_dir, clip_len=10):
        self.clip_len = clip_len
        self.clips = []
        if not os.path.exists(root_dir):
            return
        for vid in sorted(os.listdir(root_dir)):
            vid_path = os.path.join(root_dir, vid)
            if not os.path.isdir(vid_path):
                continue
            frames = sorted(glob(os.path.join(vid_path, "*.jpg")))
            for i in range(0, len(frames) - clip_len + 1, clip_len):
                self.clips.append(frames[i:i + clip_len])

    def __len__(self):
        return len(self.clips)

    def __getitem__(self, idx):
        return self.clips[idx]


def extract_object_tubes(frame_paths, confidence=0.4, augment=False):
    """Extract object tubes from frames"""
    T = len(frame_paths)
    mid_idx = T // 2 if T >= 4 else -1
    
    # Detect objects on middle frame (cached)
    areas = detect_objects_cached(frame_paths[mid_idx], confidence=confidence, coi=[0])
    if not areas:
        return None
    
    img = cv2.imread(frame_paths[0])
    H, W = img.shape[:2]
    areas = get_resized_area(areas, max_w=W, max_h=H, factor_x=FACTOR_X, factor_y=FACTOR_Y)
    
    if not areas:
        return None
    
    # Load all frames once
    frames = [cv2.cvtColor(cv2.imread(p), cv2.COLOR_BGR2RGB) for p in frame_paths]
    N_obj = len(areas)
    tubes = torch.zeros((N_obj, 3, T, OBJ_SIZE[0], OBJ_SIZE[1]))
    
    for obj_idx, bbox in enumerate(areas):
        for t, frame in enumerate(frames):
            tubes[obj_idx, :, t, :, :] = crop_and_normalize(frame, bbox, OBJ_SIZE[0], OBJ_SIZE[1], augment=augment)
    
    return tubes

In [12]:
# ============================================================
# TRAINING PIPELINE
# ============================================================

def train_pipeline():
    """Build memory banks from training data"""
    if os.path.exists("SM_prime.pt"):
        print("Memory banks exist. Skipping training.")
        return

    print(f"\n--- Training on {TRAIN_PATH} ---")
    dataset = ClipDataset(TRAIN_PATH, CLIP_LEN)
    loader = DataLoader(dataset, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, 
                       shuffle=False, pin_memory=True)
    
    SM, TM, GM = [], [], []

    for idx, batch in enumerate(tqdm(loader, desc='Training')):
        paths = [batch[i][0] for i in range(CLIP_LEN)]
        
        # Local stream
        tubes = extract_object_tubes(paths, confidence=0.4, augment=True)
        if tubes is not None:
            N, C, T, H, W = tubes.shape
            x = tubes.permute(0, 2, 1, 3, 4).reshape(N * T, C, H, W)
            x = F.interpolate(x, size=(224, 224), mode='bilinear', align_corners=False).to(device)
            
            with torch.amp.autocast('cuda'):
                fmaps = vpc(x)
                local_feat = vpc.make_locally_aware_feature(fmaps, T)
                SM.append(MakeSpatialFeature(local_feat).cpu())
                TM.append(MakeTemporalFeature(local_feat, mode='train').cpu())
        
        # Global stream
        global_imgs = torch.stack([load_frame_cv2(p, 224, 224, augment=True) for p in paths]).to(device)
        with torch.amp.autocast('cuda'):
            gfmaps = vpc(global_imgs)
            global_feat = vpc.make_globally_aware_feature(gfmaps)
            GM.append(MakeHighlevelFeature(global_feat).cpu())

    # Build and save memory banks
    if SM:
        SM, TM = torch.cat(SM), torch.cat(TM)
        print(f"\nReducing Spatial: {len(SM)} -> 3000")
        torch.save(get_coreset_fast(SM, 3000), "SM_prime.pt")
        print(f"Reducing Temporal: {len(TM)} -> 2000")
        torch.save(get_coreset_fast(TM, 2000), "TM_prime.pt")
    
    if GM:
        GM = torch.cat(GM)
        print(f"Reducing Global: {len(GM)} -> 1000")
        torch.save(get_coreset_fast(GM, 1000), "HSM_prime.pt")
    
    print("Training Complete.")

In [13]:
# ============================================================
# INFERENCE PIPELINE
# ============================================================

def inference_pipeline():
    """Compute anomaly scores for test data"""
    print(f"\n--- Inference on {TEST_PATH} ---")
    
    SMp = torch.load("SM_prime.pt").to(device)
    TMp = torch.load("TM_prime.pt").to(device)
    HSMp = torch.load("HSM_prime.pt").to(device)

    dataset = ClipDataset(TEST_PATH, CLIP_LEN)
    loader = DataLoader(dataset, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, 
                       shuffle=False, pin_memory=True)
    
    scores = []
    
    for idx, batch in enumerate(tqdm(loader, desc='Inference')):
        paths = [batch[i][0] for i in range(CLIP_LEN)]
        
        LAS = 0.0
        tubes = extract_object_tubes(paths, confidence=0.4, augment=False)
        
        if tubes is not None:
            N, C, T, H, W = tubes.shape
            x = tubes.permute(0, 2, 1, 3, 4).reshape(N * T, C, H, W)
            x = F.interpolate(x, size=(224, 224), mode='bilinear', align_corners=False).to(device)
            
            with torch.amp.autocast('cuda'):
                fmaps = vpc(x)
                local_feat = vpc.make_locally_aware_feature(fmaps, T)
                sp = MakeSpatialFeature(local_feat).to(device)
                tm = MakeTemporalFeature(local_feat, mode='test').to(device)
                ds = predict(sp, SMp, POWER_N)
                dt = predict(tm, TMp, POWER_N)
                LAS = 0.7 * ds + 0.3 * dt

        # Global stream (augment=True preserved from original)
        global_imgs = torch.stack([load_frame_cv2(p, 224, 224, augment=True) for p in paths]).to(device)
        with torch.amp.autocast('cuda'):
            gfmaps = vpc(global_imgs)
            global_feat = vpc.make_globally_aware_feature(gfmaps)
            highlevel = MakeHighlevelFeature(global_feat).to(device)
            dg = predict(highlevel, HSMp, POWER_N)

        final = (0.7 * LAS + 0.3 * dg).cpu().item()
        scores.append(final)

    # Normalize scores
    scores = torch.tensor(scores)
    scores = (scores - scores.min()) / (scores.max() - scores.min() + 1e-6)
    torch.save(scores, "frame_scores.pt")
    print("Inference Complete.")

In [14]:
# ============================================================
# GENERATE CSV
# ============================================================

def generate_csv():
    """Generate submission CSV"""
    print("\n--- Generating Submission CSV ---")
    if not os.path.exists("frame_scores.pt"):
        print("No scores found!")
        return
    
    clip_scores = torch.load("frame_scores.pt").numpy()
    video_folders = sorted(os.listdir(TEST_PATH))
    
    submission_rows = []
    score_ptr = 0
    
    for vid_folder in video_folders:
        vid_path = os.path.join(TEST_PATH, vid_folder)
        if not os.path.isdir(vid_path):
            continue
        
        frames = sorted(glob(os.path.join(vid_path, "*.jpg")))
        num_frames = len(frames)
        num_clips = num_frames // CLIP_LEN
        
        if num_clips == 0:
            vid_scores = [0.0] * num_frames
        else:
            c_scores = clip_scores[score_ptr:score_ptr + num_clips]
            score_ptr += num_clips
            vid_scores = np.repeat(c_scores, CLIP_LEN)
            remaining = num_frames - len(vid_scores)
            if remaining > 0:
                vid_scores = np.concatenate([vid_scores, [c_scores[-1]] * remaining])

        try:
            vid_id = int(vid_folder)
        except ValueError:
            vid_id = vid_folder

        for i, f_path in enumerate(frames):
            f_name = os.path.basename(f_path).split('.')[0]
            f_num = int("".join(filter(str.isdigit, f_name)))
            row_id = f"{vid_id}_{f_num}"
            submission_rows.append([row_id, vid_scores[i]])

    df = pd.DataFrame(submission_rows, columns=['Id', 'Predicted'])
    df.to_csv(OUTPUT_FILE, index=False)
    print(f"Saved {OUTPUT_FILE} with {len(df)} rows.")
    print(df['Predicted'].describe())

In [15]:
# ============================================================
# MAIN - RUN EVERYTHING
# ============================================================

if __name__ == "__main__":
    print("="*60)
    print("VIDEOPATCHCORE - OPTIMIZED")
    print("="*60)
    
    train_pipeline()
    inference_pipeline()
    generate_csv()
    
    print("\n" + "="*60)
    print("COMPLETE!")
    print("="*60)

VIDEOPATCHCORE - OPTIMIZED

--- Training on /kaggle/input/pixel-play-26/Avenue_Corrupted-20251221T112159Z-3-001/Avenue_Corrupted/Dataset/training_videos ---


Training: 100%|██████████| 913/913 [08:56<00:00,  1.70it/s]



Reducing Spatial: 359758 -> 3000


Reducing Temporal: 66078 -> 2000


Reducing Global: 9130 -> 1000


Training Complete.

--- Inference on /kaggle/working/cleaned_testing_videos ---


Inference: 100%|██████████| 1159/1159 [07:12<00:00,  2.68it/s]

Inference Complete.

--- Generating Submission CSV ---
Saved submission.csv with 11706 rows.
count    11706.000000
mean         0.147162
std          0.136209
min          0.000000
25%          0.041920
50%          0.088116
75%          0.229660
max          1.000000
Name: Predicted, dtype: float64

COMPLETE!
